# 数据预处理
## 读取数据集

In [35]:
import os
import torch

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')

with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')


从创建的CSV文件中加载原始数据集，我们导入pandas包并调用read_csv函数。

In [37]:
import pandas as pd
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值
“NaN”项代表缺失值。 为了**处理缺失的数据**，典型的方法包括**插值法**和**删除法**， 其中**插值法用一个替代值弥补缺失值**，而**删除法则直接忽略缺失值**。 在这里，我们将考虑插值法。

通过位置索引 `iloc`，我们将data分成inputs和outputs， 其中前者为data的前两列，而后者为data的最后一列。   
对于inputs中缺少的数值，我们用同一列的均值替换“NaN”项。

In [39]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
# 对数值列填充缺失值
inputs['NumRooms'] = inputs['NumRooms'].fillna(inputs['NumRooms'].astype(float).mean())
# 对非数值列填充缺失值（例如用字符串 'NA' 填充）
inputs['Alley'] = inputs['Alley'].fillna('NA')
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0    NA
2       4.0    NA
3       3.0    NA


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。   
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [41]:
inputs = pd.get_dummies(inputs, dummy_na=True)
# 这里如果想要用 get_dummies 将True或者False转换为0或者1，需要将其转换为int类型
# inputs = pd.get_dummies(inputs, dummy_na=True).astype(int)
inputs = inputs.drop(columns='Alley_nan')
print(inputs)

   NumRooms  Alley_NA  Alley_Pave
0       3.0     False        True
1       2.0      True       False
2       4.0      True       False
3       3.0      True       False


## 转换为张量格式
转换为张量格式。 当数据采用张量格式后，可以通过在 L1 节中引入的那些张量函数来进一步操作。

In [43]:
# X的值就是inputs的值，Y的值就是outputs的值，分别对应第一列第二列和第三列
X = torch.tensor(inputs.to_numpy(dtype=float))
Y = torch.tensor(outputs.to_numpy(dtype=float))
print(X, "\n", Y)

tensor([[3., 0., 1.],
        [2., 1., 0.],
        [4., 1., 0.],
        [3., 1., 0.]], dtype=torch.float64) 
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64)
